In [27]:
import numpy as np

In [32]:
def generate_positive_definite_matrix(size: int) -> np.ndarray:
    """
    Generate random symmetric positive definite matrix:
    1. Create random matrix A
    2. Compute A^T * A (always positive semi-definite)
    3. Add diagonal dominance to ensure positive definite
    """
    rng = np.random.default_rng(52)
    
    # Create random matrix
    A = rng.random((size, size))
    
    # Make symmetric positive definite
    matrix = A @ A.T  # Ensures positive semi-definite
    
    # Add to diagonal to ensure positive definite
    matrix += size//2 * np.eye(size)
    
    return matrix

In [35]:
mat_size = 3
mat = generate_positive_definite_matrix(3)
# for i in range(mat_size):
#   for j in range(i, mat_size):
#     mat[i, j] = mat[j, i]

print(mat)
# mat = np.ndarray((2, 2), dtype=np.int64)


def inverse_iteration(A: np.ndarray, tolerance: np.float64 = 1e-8, max_iter: int = 500):
    dim = A.shape[0]

    # initial guess
    x_prev = np.random.rand(dim)
    # print(x_prev)

    sigma = 0
    sigma_prev = 0

    # decompose A into L*U using cholesky
    L = np.linalg.cholesky(A, upper=False)
    U = L.T

    # U = np.linalg.cholesky(A, upper=True)

    # print(L @ L.T) # A matr

    for _ in range(max_iter):
        norm = np.linalg.norm(x_prev, ord=None)
        # print(norm)
        nu = x_prev / norm # +- eigenvec

        # L*y = nu
        # U*x = y
        y = np.linalg.solve(L, nu)
        x_next = np.linalg.solve(U, y)

        sigma_prev = sigma
        sigma = 1 / np.matmul(y, x_next)  # eigenvalue

        if abs(sigma - sigma_prev) < tolerance and abs(sigma - sigma_prev) != 0:
          print(_)
          return nu, sigma

        sigma_prev = sigma
        x_prev = x_next

    return nu, sigma

eigvec_1, eigval_1 = inverse_iteration(mat)

print(eigval_1, eigvec_1)

[[1.97882539 0.82302082 0.38084467]
 [0.82302082 2.24054186 0.24729345]
 [0.38084467 0.24729345 1.21811758]]
59
1.086770456781609 [-0.49198718  0.16186474  0.85542295]


In [40]:
def inverse_iteration_with_shift_one(A: np.ndarray, tolerance: np.float64 = 1e-8, max_iter: int = 500):
    dim = A.shape[0]

    # initial guess
    x_prev = np.random.rand(dim)
    # print(x_prev)

    sigma = 0
    sigma_prev = 0

    # decompose A into L*U using cholesky
    L = np.linalg.cholesky(A, upper=False)
    U = L.T

    g_1, _ = inverse_iteration(A)
    print(g_1, _)
    
    # U = np.linalg.cholesky(A, upper=True)

    # print(L @ L.T) # A matr

    for _ in range(max_iter):
        norm = np.linalg.norm(x_prev, ord=None)
        # print(norm)
        nu = x_prev / norm # +- eigenvec

        # f = E - g_1*g_1^T
        # L*y = f * nu
        # U*x = y
        y = np.linalg.solve(L, np.matmul(np.eye(dim) - np.matmul(g_1, g_1.T), nu))
        x_next = np.linalg.solve(U, y)

        sigma_prev = sigma
        sigma = 1 / np.matmul(y, x_next)  # eigenvalue

        if abs(sigma - sigma_prev) < tolerance and abs(sigma - sigma_prev) != 0:
          print(_)
          return nu, sigma

        sigma_prev = sigma
        x_prev = x_next

    return nu, sigma

In [50]:
print(inverse_iteration_with_shift_one(mat))

66
[-0.49198723  0.16186481  0.8554229 ] 1.0867704589974014
152
(array([ 0.7363108 , -0.25770109, -0.62564891]), np.float64(1.2405279824854132))
